**SOW-MKI49: Neural Information Processing Systems**  
*Weeks 6 and 7: Assignment (100 points + 20 bonus points)  
Author: Luca and Umut

# Group number: 17

1. Sameera Sandaruwan, s1014012
2. Mohit Jethwani, s1019474

# To Implement

1. Encoder - DONE
2. Decoder - DONE
3. Loss function - DONE
4. Model (sample from latent features) - DONE
5. Data preparation - Mohit
6. Training block - Mohit

# Questions to ask


# Notes
---
http://kvfrans.com/variational-autoencoders-explained/

However, there were a couple of downsides to using a plain GAN.

First, the images are generated off some arbitrary noise. If you wanted to generate a picture with specific features, there's no way of determining which initial noise values would produce that picture, other than searching over the entire distribution.

Second, a generative adversarial model only discriminates between "real" and "fake" images. There's no constraints that an image of a cat has to look like a cat. This leads to results where there's no actual object in a generated image, but the style just looks like picture.

In this post, I'll go over the variational autoencoder, a type of network that solves these two problems.

The more efficiently we can encode the original image, the higher we can raise the standard deviation on our gaussian until it reaches one.

https://github.com/chainer/chainer/blob/master/examples/vae/net.py

https://github.com/musyoku/variational-autoencoder/blob/master/vae_m1.py

---
## To Do

1. Search Variational autoencoder
2. Search Variational inference
3. Find Input and Output dimensions of dataset: 
Depends on our Network architecture. Network can be Convolutional (input is 28x28, output 1x10) or Linear (input is 1x784, output 1x10)
4. Check the architecture
5. How the loss function work here
6. figure out the architecture and what (X, Y, Z) are
7. Unit gaussian distribution

sdf

In [1]:
from chainer import cuda, datasets, serializers
from chainer.dataset import DatasetMixin, concat_examples
from chainer.iterators import MultithreadIterator
import chainer
import chainer.functions as F
import chainer.links as L
import cupy
import numpy as np
import tqdm

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
n_in = 28*28
n_h = 1024
n_latent = 10

# Decoder
This is the decoder class. It transforms **latents (features) to observables (images)**. It corresponds to p(x | z) in the context of **variational inference** (and the slides), **where x is observables and y is latents.**

Task: (10 points)

- Implement the decoder class for a **variational autoencoder**. Note that the **decoder** should output the **Gaussian distribution parameters (mean and variance per pixel)** of images rather than images themselves.

In [0]:
class Decoder(chainer.ChainList):
    def __init__(self): # <= you might want to pass some architecture parameters (e.g., #i/o units, etc.) here
        super(Decoder, self).__init__(
            # pass your decoder layers here
            ly1 = L.Linear(10, 1024),
            mean_x = L.Linear(1024, 784),
            ln_var_x = L.Linear(1024, 784),
        )

    def __call__(self, x):
        h = F.tanh(self.ly1(x))
        mean_x = self.mean_x(h)
        ln_var_x = self.ln_var_x(h)
        
        return mean_x, ln_var_x

# Encoder
This is the encoder class. It transforms **observables (images) to latents (features)**. It corresponds to **q(z|x)** in the context of variational inference (and the slides), **where z is latents and x is observables**.

Task: (10 points)

- Implement the encoder class for a variational autoencoder. Note that the encoder should output the Gaussian distribution parameters (mean and variance per feature) of features rather than features themselves.

In [0]:
class Encoder(chainer.ChainList):
    def __init__(self): # <= you might want to pass some architecture parameters (e.g., #i/o units, etc.) here
        super(Encoder, self).__init__(
            # pass your encoder layers here
            ly1 = L.Linear(784, 1024),
            mean_z = L.Linear(1024, 10),
            ln_var_z = L.Linear(1024, 10),
        )

    def __call__(self, x):
        h = F.tanh(self.ly1(x))
        mean_z = self.mean_z(h)
        ln_var_z = self.ln_var_z(h)
        
        return mean_z, ln_var_z
'''
https://www.kaggle.com/luyujia/mnist-chainer-cnn#
check this - https://github.com/musyoku/variational-autoencoder/blob/master/vae_m1.py

Convolutional Encoder  - backup
conv1 = L.Convolution2D(in_channels=1, out_channels=32, ksize=5),
conv2 = L.Convolution2D(in_channels=32, out_channels=64, ksize=5),
mean_z = L.Linear(1024, 10),
ln_var_z = L.Linear(1024, 10),

'''

# Loss
This is the loss class. The loss of encoder and decoder of a variational autoencoder is the evidence lower bound as follows:

$L = D_{KL}(q(z | x), p(z)) -  E_{z\sim q}[log~p(x | z)]$

The first term above is the KL divergence between the approximate posterior (q) and the prior (p), which can be interpreted as a form of regularization. You can assume that the prior is unit Gaussian. It can be implemented with the F.gaussian_kl_divergence function in Chainer.

The second term above is the Gaussian negative log likelihood. This is the term that fits the data, which is very similar to the usual loss functions that you use in deep learning. It can be implemented with the F.gaussian_nll function in Chainer.

Task: 

- Implement the loss class. (10 points)


As input, it gets the following arguments:

mean_z => mean of the encoded features (output of the encoder)  
ln_var_z => log variance of the encoded features (output of the encoder)  
x => input images (mini batch)  
mean_x => mean of the decoded images (output of the decoder)  
ln_var_x => mean of the decoded images (output of the decoder)  

As output, it gives the loss.

- Explain why we use log variance instead of variance. (5 points)

**Answer**


In [ ]:
'''
# z_mean and z_stddev are two vectors generated by encoder network
latent_loss = 0.5 * tf.reduce_sum(tf.square(z_mean) + tf.square(z_stddev) - tf.log(tf.square(z_stddev)) - 1,1) 
'''

In [1]:
# check this - https://github.com/musyoku/variational-autoencoder/blob/master/vae_m1.py
# https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
# https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf
class Loss(object):
    def __call__(self, mean_z, ln_var_z, x, mean_x, ln_var_x):
        # Implement the loss here     
        '''
        1. KL divergence
        F.gaussian_kl_divergence
        
        𝐷𝐾𝐿(𝑞(𝑧|𝑥),𝑝(𝑧))
        
        ''' 
        dkl = F.gaussian_kl_divergence(mean_z, ln_var_z)
        '''
        2. Gaussian negative log likelihood
        F.gaussian_nll
        
        𝐸𝑧∼𝑞[𝑙𝑜𝑔(𝑝(𝑥|𝑧))]
        
        '''
        Gnll = F.gaussian_nll(x, mean_x, ln_var_x)
        
        return dkl - Gnll
        

# Model
This is the model class. It combines the encoder and the decoder.

Task: (20 points)

- Implement the reparameterziation trick for sampling latents. (10 points)



- Explain why we need to use this trick. (10 points)

**The latent loss, which is the KL divergence that measures how closely the latent variables match a unit gaussian. In order to optimize the KL divergence, we need the trick: instead of the encoder generating a vector of real values, it will generate a vector of means and a vector of standard deviations.**

In [0]:
class Model(chainer.Chain):
    def __init__(self, decoder, encoder):
        super(Model, self).__init__()

        with self.init_scope():
            self.decoder = decoder
            self.encoder = encoder

    def __call__(self, x):
        mean_z, ln_var_z = self.encoder(x)
        
        # Sample latents (z) from the Gaussian with parameters ln_var_z, mean_z by using the reparameterization trick
        '''
        we can just sample from the standard deviations and add the mean, and use that as our latent vector: 
        samples = tf.random_normal([batchsize,n_z],0,1,dtype=tf.float32)  
        sampled_z = z_mean + (z_stddev * samples)
        '''
        z = np.random.normal(mean_z, ln_var_z, np.broadcast(mean_z, ln_var_z).size)
        mean_x, ln_var_x = self.decoder(z)

        return mean_z, ln_var_z, x, mean_x, ln_var_x

This is a helper class to use the Mnist dataset.

In [0]:
class Mnist(DatasetMixin):
    def __init__(self):
        self.dataset = datasets.get_mnist(False)[0 if chainer.config.train else 1]

    def __len__(self):
        return len(self.dataset)

    def get_example(self, i):
        return self.dataset[i]

Task: (50 points)

- Train the above defined variational autoencoder on the Mnist dataset. You can refer to the earlier assignments to implement your training loop. (25 points)

- How good are the samples? Randomy sample some digits and visualize them. (10 points)

- How good are the reconstructions? Draw an Mnist like digit, encode it, decode it and visualize the digits. How different is the reconstruction from the original. (10 points)

- Repeat the last task but by drawing something other than a digit (e.g., a face). How accuracte is the reconstructions? Explain the results. (5 points)

# Training

In [ ]:
### Init model
device = 0
encoder = Encoder()
decoder = Decoder()
model = Model(decoder, encoder) if device < 0 else Model(decoder, encoder).to_gpu(device)

#optimizer = Adam(alpha=0.001, beta1=0.9, beta2=0.999, eps=1e-08)
optimizer = Adam()
optimizer.setup(model)

### Prep dataset


Bonus task: Try the same experiments on a different dataset. (20 bonus points)